In [3]:
import requests
import time
from requests.compat import urljoin
from bs4 import BeautifulSoup
from PIL import Image
import io
import shutil
import os

In [69]:
dirname = "data"
ims = []
for filename in os.listdir(dirname) :
    fullname = os.path.join(dirname, filename)
    #print(fullname)
    im = Image.open(fullname)
    #display(im)
    ims.append(im)

In [70]:
print(ims[0].size)

rects = [(17,12,35,35),(37,10,53,33),(56,10,72,33),(76,12,92,35)]

digit1 = []
digit2 = []
digit3 = []
digit4 = []

for im in ims:
    # split first digit
    digit1.append(im.copy().crop(rects[0]))
    digit2.append(im.copy().crop(rects[1]))
    digit3.append(im.copy().crop(rects[2]))
    digit4.append(im.copy().crop(rects[3]))
    #display(digit1[-1])

(100, 40)


In [71]:
import numpy as np

digit1[-1].getbands()

np_digit1 = []
for im in digit1:
    np_digit1.append( np.array(im.convert("L").getdata()) )
    
#np_digit1[-1]

In [72]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=20,random_state=22)
kmeans.fit(np_digit1)

KMeans(n_clusters=20, random_state=22)

In [73]:
print(kmeans.labels_)

groups = {}
for file, cluster in zip(digit1,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

#print(groups)
for g in groups:
    print(g, len(groups[g]))
    
for im in groups[4]:
    display(im)

[14  4  6  3 13  4 10  5 15 18 11  1 19 12 16  1 17  2  9  7  8  0]
14 1
4 2
6 1
3 1
13 1
10 1
5 1
15 1
18 1
11 1
1 2
19 1
12 1
16 1
17 1
2 1
9 1
7 1
8 1
0 1
